In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
os.chdir('/content/drive/MyDrive/Seed_Classification_4_varieties/resnet1D')
os.getcwd()

'/content/drive/MyDrive/Seed_Classification_4_varieties/resnet1D'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imblearn.over_sampling as oversample
from  model import *
from  train_eval import *
import torch
from torchsummary import summary
import torch.optim as optim
import pickle
from sklearn.metrics import confusion_matrix, classification_report


In [ ]:
BASE_DIR = {
    'crease_up' : '/content/drive/MyDrive/Seed_Classification_4_varieties/crease_up/Data' ,
    'crease_down' : '/content/drive/MyDrive/Seed_Classification_4_varieties/crease_down/Data' ,
    'crease_up_down_combined' : '/content/drive/MyDrive/Seed_Classification_4_varieties/crease_up_down_combined/Data'
}
config = {
    'BATCH_SIZE' : 64,
    'lr' : 0.000008,
    'EPOCHS' : 50,
    'input_size' : 147,
    'output_size' : 4,

    'val_path' : 'Data/df_val.csv',
    'tst_path' : 'Data/df_tst.csv' ,
}

SAVE_DIR = '/content/drive/MyDrive/Seed_Classification_4_varieties/resnet1D'
def update_config(data_type:str):
  config['val_path'] = f"{BASE_DIR[data_type]}/df_val.csv"
  config['tst_path'] = f"{BASE_DIR[data_type]}/df_tst.csv"
  config['input_size'] = 294 if data_type == 'crease_up_down_combined' else 147

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# df_tst = pd.read_csv(config['tst_path'])

In [ ]:
model = ResNet1D(num_classes = config['output_size'] , input_length=config['input_size']).to(device)
summary(model, (config['input_size'],))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1               [-1, 16, 74]             128
       BatchNorm1d-2               [-1, 16, 74]              32
             PReLU-3               [-1, 16, 74]               1
         MaxPool1d-4               [-1, 16, 37]               0
            Conv1d-5               [-1, 32, 19]           1,568
       BatchNorm1d-6               [-1, 32, 19]              64
             PReLU-7               [-1, 32, 19]               1
            Conv1d-8               [-1, 32, 19]           3,104
       BatchNorm1d-9               [-1, 32, 19]              64
           Conv1d-10               [-1, 32, 19]             544
      BatchNorm1d-11               [-1, 32, 19]              64
            PReLU-12               [-1, 32, 19]               1
  ResidualBlock1D-13               [-1, 32, 19]               0
           Conv1d-14               [-1,

## Plotting


In [ ]:
def plot_history(history:dict, model_name):
      plt.figure( figsize=(20,10))

    #   plt.subplot(2, 1, 1)
      plt.plot(history['train_loss'], 'o-', color='green')
      plt.plot(history['val_loss'], 'o-', color='red')
      plt.plot(history['train_acc'], 'o-', color='pink')
      plt.plot(history['val_acc'], 'o-', color='blue')
    #   plt.ylabel('Loss')
      plt.xlabel('Epochs', fontsize = 14)
      plt.legend(['Train Loss' , 'Validation Loss','Train Accuracy' , 'Validation Accuracy'],  fontsize = 14)


      plt.savefig('{x}_Train_Val_Curves.png'.format(x = model_name), bbox_inches='tight')


## Imbalanced Data Training

In [ ]:
for data_type in ['crease_up', 'crease_down', 'crease_up_down_combined']:
    update_config(data_type)

    config['tr_path'] = f'{BASE_DIR[data_type]}/df_tr_imbalanced.csv'
    config['name'] = f'{data_type}_imbalanced'

    df_tr = pd.read_csv(config['tr_path'])
    df_tst = pd.read_csv(config['tst_path'])

    # Using iloc for positional indexing
    y = np.array(df_tr.iloc[:, -1])  # Target variable (last column)

    # Using iloc for the features
    mean = torch.tensor(np.mean(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)
    std = torch.tensor(np.std(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)

    print(mean.shape, std.shape)

    if not os.path.exists(f'{SAVE_DIR}/{data_type}_imbalanced_model.pt'):
      tr_dataset = MyDataset(path =config ['tr_path'] , mean = mean ,std = std , apply_transform = True)
      tr_loader = DataLoader(tr_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      val_dataset = MyDataset(path = config['val_path'] , mean = mean ,std = std, apply_transform = True)
      val_loader = DataLoader(val_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      X,Y = next(iter(tr_loader))
      print(X.shape , Y.shape)

      model = ResNet1D(num_classes = config['output_size'] , input_length=config['input_size']).to(device)
      optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
      criterion = nn.CrossEntropyLoss()    # weight = class_weights.to(device)

      # lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5,cooldown= 2, factor = 0.4, verbose = True, threshold = 1e-2)

      lr_scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.975, last_epoch=-1)

      obj = Train_Eval(model=model, model_name=config['name'] ,device=device ,train_loader=tr_loader,
                    val_loader=val_loader, optimizer=optimizer, criterion=criterion, lr_scheduler= lr_scheduler)
      obj.run(config['EPOCHS'])
      obj.eval(df_tst, mean, std)

      del model
      del optimizer
      del criterion
      del lr_scheduler
      del obj
      torch.cuda.empty_cache()


torch.Size([147]) torch.Size([147])
torch.Size([147]) torch.Size([147])
torch.Size([294]) torch.Size([294])


## SMOTE-Balanced Training

In [ ]:
for data_type in ['crease_up', 'crease_down', 'crease_up_down_combined']:
    update_config(data_type)

    config['tr_path'] = f'{BASE_DIR[data_type]}/df_tr_smote.csv'
    config['name'] = f'{data_type}_smote'

    df_tr = pd.read_csv(config['tr_path'])
    df_tst = pd.read_csv(config['tst_path'])
    # Using iloc for positional indexing
    y = np.array(df_tr.iloc[:, -1])  # Target variable (last column)

    # Using iloc for the features
    mean = torch.tensor(np.mean(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)
    std = torch.tensor(np.std(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)

    print(mean.shape, std.shape)

    if not os.path.exists(f'{SAVE_DIR}/{data_type}_smote_model.pt'):
      tr_dataset = MyDataset(path =config ['tr_path'] , mean = mean ,std = std , apply_transform = True)
      tr_loader = DataLoader(tr_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      val_dataset = MyDataset(path = config['val_path'] , mean = mean ,std = std, apply_transform = True)
      val_loader = DataLoader(val_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      X,Y = next(iter(tr_loader))
      print(X.shape , Y.shape)

      model = ResNet1D(num_classes = config['output_size'] , input_length=config['input_size']).to(device)
      optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
      criterion = nn.CrossEntropyLoss()    # weight = class_weights.to(device)

      # lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5,cooldown= 2, factor = 0.4, verbose = True, threshold = 1e-2)

      lr_scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99, last_epoch=-1)

      obj = Train_Eval(model=model, model_name=config['name'] ,device=device ,train_loader=tr_loader,
                    val_loader=val_loader, optimizer=optimizer, criterion=criterion, lr_scheduler= lr_scheduler)
      obj.run(config['EPOCHS'])
      obj.eval(df_tst, mean, std)

      del model
      del optimizer
      del criterion
      del lr_scheduler
      del obj
      torch.cuda.empty_cache()


torch.Size([147]) torch.Size([147])
torch.Size([147]) torch.Size([147])
torch.Size([294]) torch.Size([294])
torch.Size([64, 294]) torch.Size([64])

Epoch: 0


100%|██████████| 64/64 [00:01<00:00, 60.84it/s]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:232: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [0] --> LossTr: 1.2364    AccTr: 0.5469    lossVal : 1.1041     accVal : 0.6611

Detected network improvement, saving current model  "✅"

Epoch: 1


100%|██████████| 64/64 [00:01<00:00, 61.35it/s]


Epoch [1] --> LossTr: 1.0542    AccTr: 0.7180    lossVal : 1.0068     accVal : 0.7490

Detected network improvement, saving current model  "✅"

Epoch: 2


...

Epoch [47] --> LossTr: 0.7770    AccTr: 0.9684    lossVal : 0.7914     accVal : 0.9556


Epoch: 48


100%|██████████| 64/64 [00:01<00:00, 59.37it/s]


Epoch [48] --> LossTr: 0.7771    AccTr: 0.9684    lossVal : 0.7924     accVal : 0.9504


Epoch: 49


100%|██████████| 64/64 [00:01<00:00, 37.55it/s]


Epoch [49] --> LossTr: 0.7764    AccTr: 0.9686    lossVal : 0.7903     accVal : 0.9546

acc : 0.951733   
{0: 3030, 1: 3030, 2: 1414, 3: 606}


## Adasyn-Data Training

In [ ]:
for data_type in ['crease_up', 'crease_down', 'crease_up_down_combined']:
    update_config(data_type)

    config['tr_path'] = f'{BASE_DIR[data_type]}/df_tr_adasyn.csv'
    config['name'] = f'{data_type}_adasyn'

    df_tr = pd.read_csv(config['tr_path'])
    df_tst = pd.read_csv(config['tst_path'])
    # Using iloc for positional indexing
    y = np.array(df_tr.iloc[:, -1])  # Target variable (last column)

    # Using iloc for the features
    mean = torch.tensor(np.mean(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)
    std = torch.tensor(np.std(df_tr.iloc[:, :-1].values, axis=0), dtype=torch.float32)

    print(mean.shape, std.shape)

    if not os.path.exists(f'{SAVE_DIR}/{data_type}_adasyn_model.pt'):
      tr_dataset = MyDataset(path =config ['tr_path'] , mean = mean ,std = std , apply_transform = True)
      tr_loader = DataLoader(tr_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      val_dataset = MyDataset(path = config['val_path'] , mean = mean ,std = std, apply_transform = True)
      val_loader = DataLoader(val_dataset, batch_size=config['BATCH_SIZE'], shuffle=True)

      X,Y = next(iter(tr_loader))
      print(X.shape , Y.shape)

      model = ResNet1D(num_classes = config['output_size'] , input_length=config['input_size']).to(device)
      optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'])
      criterion = nn.CrossEntropyLoss()    # weight = class_weights.to(device)

      # lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 5,cooldown= 2, factor = 0.4, verbose = True, threshold = 1e-2)

      lr_scheduler=torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.99, last_epoch=-1)

      obj = Train_Eval(model=model, model_name=config['name'] ,device=device ,train_loader=tr_loader,
                    val_loader=val_loader, optimizer=optimizer, criterion=criterion, lr_scheduler= lr_scheduler)
      obj.run(config['EPOCHS'])
      obj.eval(df_tst, mean, std)

      del model
      del optimizer
      del criterion
      del lr_scheduler
      del obj
      torch.cuda.empty_cache()


torch.Size([147]) torch.Size([147])
torch.Size([147]) torch.Size([147])
torch.Size([294]) torch.Size([294])
